In [2]:
import warnings
warnings.filterwarnings("ignore")

from time import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import pandas as pd

from imblearn.over_sampling import SMOTE
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier  
from sklearn.metrics import confusion_matrix
import seaborn as sns

from sklearn.metrics import classification_report

from sklearn.model_selection import cross_val_predict

pd.set_option('display.max_rows', None)

In [3]:
df_crisis_boe = pd.read_excel('/content/sample_data/Bank_ready.xlsx')

df_crisis_mean = pd.read_excel('/content/sample_data/Mean_ready.xlsx') 

df_crisis_median = pd.read_excel('/content/sample_data/Median_ready.xlsx') 

df_crisis_mostf = pd.read_excel('/content/sample_data/Most_f_ready.xlsx')  

In [4]:
def datapreparation(df_crisis):

  dfx_crisis = df_crisis.drop(['year','iso'],axis=1) #retirar atributos sem relação a crises
  
  dfx_crisis = dfx_crisis.drop(['crisis'],axis=1) #retirar variavel binaria indicadora de crises

  dfy_crisis = df_crisis['crisis'].copy() 
  dfx_crisis.shape, dfy_crisis.shape
  
  dfx_crisis_resampled, dfy_crisis_resampled = SMOTE().fit_resample(dfx_crisis, dfy_crisis)
  dfx_crisis_resampled,dfy_crisis_resampled = shuffle(dfx_crisis_resampled,dfy_crisis_resampled)

  dfx_crisis_resampled = pd.DataFrame(dfx_crisis_resampled)
  dfy_crisis_resampled = pd.DataFrame(dfy_crisis_resampled)
  dfx_crisis_resampled.shape, dfy_crisis_resampled.shape

  return dfx_crisis_resampled, dfy_crisis_resampled

In [5]:
dfx_boe, dfy_boe = datapreparation(df_crisis_boe)
dfx_mean, dfy_mean = datapreparation(df_crisis_mean)
dfx_median, dfy_median = datapreparation(df_crisis_median)
dfx_mostf, dfy_mostf = datapreparation(df_crisis_mostf)

In [6]:
dfx_boe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2308 entries, 0 to 2307
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       2308 non-null   float64
 1   1       2308 non-null   float64
 2   2       2308 non-null   float64
 3   3       2308 non-null   float64
 4   4       2308 non-null   float64
 5   5       2308 non-null   float64
 6   6       2308 non-null   float64
 7   7       2308 non-null   float64
 8   8       2308 non-null   float64
 9   9       2308 non-null   float64
 10  10      2308 non-null   float64
 11  11      2308 non-null   float64
dtypes: float64(12)
memory usage: 216.5 KB


In [7]:
dfy_boe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2308 entries, 0 to 2307
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       2308 non-null   int64
dtypes: int64(1)
memory usage: 18.2 KB


In [8]:
# Split the dataset in two equal parts
def spliter(dfx,dfy):
  X_train, X_test, y_train, y_test = train_test_split(dfx, dfy, test_size=0.75, random_state=0)
  return X_train, X_test, y_train, y_test

In [9]:
# Set the parameters by cross-validation
# Possibilidades de testes para quantidade de neurônios e camadas
# 'hidden_layer_sizes': [10,10,10,10,10], [20,20,20,20,20], [30,30,30,30,30],
# [40,40,40,40,40], [50,50,50,50,50], [60,60,60,60,60], [70,70,70,70,70], 
# [80,80,80,80,80], [90,90,90,90,90], [100,100,100,100,100]
# [10,10,10,10,10,10,10,10,10,10], [20,20,20,20,20,20,20,20,20,20],
# [30,30,30,30,30,30,30,30,30,30], [40,40,40,40,40,40,40,40,40,40],
# [50,50,50,50,50,50,50,50,50,50], [60,60,60,60,60,60,60,60,60,60],
# [70,70,70,70,70,70,70,70,70,70],[80,80,80,80,80,80,80,80,80,80],
# [90,90,90,90,90,90,90,90,90,90], [100,100,100,100,100,100,100,100,100,100]

tuned_parameters = [{'hidden_layer_sizes': [50,50,50,50,50],
                     'activation': ['identity','logistic','tanh','relu'],
                     'alpha': [0.0001,0.00001,0.000001],
                     'solver': ['sgd', 'adam','lbfgs'],
                     'learning_rate': ['constant', 'invscaling', 'adaptive'],
                     'momentum': [0.5,0.75,0.78,0.90,0.95, 0.99],
                     'max_iter': [int(x) for x in np.linspace(start = 500, stop = 5000, num = 100)]}]

scores = ['precision', 'recall']


In [10]:
def p_estimation(dfx,dfy):
  X_train, X_test, y_train, y_test = spliter(dfx,dfy)
  for score in scores:
      print("# Tuning hyper-parameters for %s" % score)
      print()

      mlp = GridSearchCV(
          MLPClassifier(), tuned_parameters, scoring='%s_macro' % score
      )
      mlp.fit(X_train, y_train)

      print("Best parameters set found on development set:")
      print()
      print(mlp.best_params_)
      print()
      print("Grid scores on development set:")
      print()
      means = mlp.cv_results_['mean_test_score']
      stds = mlp.cv_results_['std_test_score']
      for mean, std, params in zip(means, stds, mlp.cv_results_['params']):
          print("%0.3f (+/-%0.03f) for %r"
                % (mean, std * 2, params))
      print()

      print("Detailed classification report:")
      print()
      print("The model is trained on the full development set.")
      print("The scores are computed on the full evaluation set.")
      print()
      y_true, y_pred = y_test, mlp.predict(X_test)
      print(classification_report(y_true, y_pred))
      print()

In [1]:
p_estimation(dfx_boe, dfy_boe)

NameError: ignored

In [ ]:
p_estimation(dfx_mean, dfy_mean)

In [ ]:
p_estimation(dfx_median, dfy_median)

In [ ]:
p_estimation(dfx_mostf, dfy_mostf)